In [284]:
import pandas as pd
import re
import statistics as st

#function for nice display
def summary(cost):
    df1 = pd.DataFrame()
    df1['cost'] = cost  
    mean = sum(df1['cost']) / len(df1['cost'])
    median = st.median(df1['cost'])           
    v = df1['cost'].var()
    s = v ** 0.5
    df2 = pd.DataFrame()
    df2["Values"] = ["%.2f" %mean, median,"%.2f" %v, s]
    df2.index = ['Mean', 'Median', 'Variance', 'Std Dev']   
    return df2

In [269]:
#Import the CSV with encoding ISO-8859-1 for the english '£'?
df = pd.read_csv('WELLCOME_APCspend2013_forThinkful.csv', encoding = "ISO-8859-1")
#Determine the five most common journals and the total articles for each

#See what the columns are
print(list(df))





['PMID/PMCID', 'Publisher', 'Journal title', 'Article title', 'COST (£) charged to Wellcome (inc VAT when charged)']


In [270]:
titles = df['Journal title']

#1. Force all to lower case
firststep = titles.str.lower()
#print(firststep)
print(firststep.value_counts()[0:5])

plos one                           190
journal of biological chemistry     53
neuroimage                          29
plos pathogens                      24
plos genetics                       24
Name: Journal title, dtype: int64


In [236]:
#look at some examples of what the data looks like
print(df['Journal title'].head(20))
print(df["Journal title"].value_counts()[0:30])

0                           Psychological Medicine
1                                Biomacromolecules
2                                       J Med Chem
3                                       J Med Chem
4                                       J Org Chem
5                   Journal of Medicinal Chemistry
6                     Journal of Proteome Research
7                                        Mol Pharm
8                             ACS Chemical Biology
9                             ACS Chemical Biology
10    Journal of Chemical Information and Modeling
11                  Journal of Medicinal Chemistry
12                  Journal of Medicinal Chemistry
13                                    Biochemistry
14                  Journal of Medicinal Chemistry
15                                Gastroenterology
16                 Journal of Biological Chemistry
17                           Journal of Immunology
18                           Journal of Immunology
19                            A

**Seems to be some capitalization instances that hinder getting an accurate count**

In [237]:
#Force strings to lowercase and look for more accurate grouping

#Check the first 10
print(df['Journal title'].str.lower().value_counts().head(10))

plos one                                           190
journal of biological chemistry                     53
neuroimage                                          29
plos pathogens                                      24
plos genetics                                       24
nucleic acids research                              23
proceedings of the national academy of sciences     20
plos neglected tropical diseases                    20
human molecular genetics                            19
nature communications                               19
Name: Journal title, dtype: int64


In [271]:
#The 5 most common journals would be
print(df['Journal title'].str.lower().value_counts().head(5))
#replace the df columns with lowercase entries
df.dtypes
df = df.apply(lambda x: x.astype(str).str.lower())
#print(df)
#this would still miss these entries:
print(df['Journal title'].str.lower().value_counts().head(25))




plos one                           190
journal of biological chemistry     53
neuroimage                          29
plos pathogens                      24
plos genetics                       24
Name: Journal title, dtype: int64
plos one                                           190
journal of biological chemistry                     53
neuroimage                                          29
plos genetics                                       24
plos pathogens                                      24
nucleic acids research                              23
proceedings of the national academy of sciences     20
plos neglected tropical diseases                    20
nature communications                               19
human molecular genetics                            19
brain                                               14
bmc public health                                   14
movement disorders                                  13
journal of neuroscience                             12
d

In [239]:
titlesl = df['Journal title']
tit1 = titlesl.str.replace('plosone', 'plos one')
print(tit1.value_counts().head(5))
#That's better, apply to data frame
df['Journal title'] = df['Journal title'].replace('plosone', 'plos one')
#verify
print(df['Journal title'].value_counts().head(5))
#cool



plos one                           199
journal of biological chemistry     53
neuroimage                          29
plos pathogens                      24
plos genetics                       24
Name: Journal title, dtype: int64
plos one                           199
journal of biological chemistry     53
neuroimage                          29
plos pathogens                      24
plos genetics                       24
Name: Journal title, dtype: int64


In [240]:
#check articles for duplicates
print(len(df['Article title']))
#unique values?
print(len(df['Article title'].unique()))
#assuming the spelling is correct, there is a duplicate


2127
2126


In [241]:
#drop the duplicates
df = df.drop_duplicates('Article title')
#verify
print(len(df['Article title']))

2126


In [289]:
# that £ sign with get in the way of caluclating, remove it
df['COST (£) charged to Wellcome (inc VAT when charged)'] = df['COST (£) charged to Wellcome (inc VAT when charged)'].str[1:]
#verify
print(df['COST (£) charged to Wellcome (inc VAT when charged)'].head(3))
df['COST (£) charged to Wellcome (inc VAT when charged)'] = df['COST (£) charged to Wellcome (inc VAT when charged)'].str.replace(r'$', '')


0       00
1    81.04
2     2.56
Name: COST (£) charged to Wellcome (inc VAT when charged), dtype: object


In [243]:
plosone = pd.Series(df.loc[(df['Journal title'] == 'plos one'), 'COST (£) charged to Wellcome (inc VAT when charged)'])
print(plosone)

1282      1001.03
1283      1004.15
1284      1011.45
1285      1011.45
1286      1015.73
1287      1023.41
1288      1039.87
1289      1061.24
1290      1061.24
1291      1061.24
1292      1080.00
1293       794.93
1294       809.29
1295       819.34
1296       854.96
1297       901.50
1298       902.12
1299       903.89
1303      1061.93
1304       389.73
1305       443.38
1414      1002.86
1415      1005.86
1416      1005.96
1417      1008.97
1418      1009.98
1419      1010.80
1420      1013.03
1421      1019.76
1422      1027.73
          ...    
1562    999999.00
1563    999999.00
1564    999999.00
1565    999999.00
1566    999999.00
1567       900.14
1568       904.34
1569       907.08
1570       908.01
1571       909.99
1572       910.20
1573       913.36
1574       915.53
1575       918.62
1576       953.48
1577       960.16
1578       961.68
1579       961.68
1580       961.68
1581       986.72
1582       996.06
1605       842.54
1606       842.54
1607       850.88
1608      

In [244]:
#Some of the pricing appears to be off (Cost of 99999) #let's drop those
df.iloc[1563]


PMID/PMCID                                                                        pmcid:\n    pmc3579793
Publisher                                                                      public library of science
Journal title                                                                                   plos one
Article title                                          phagocytosis is the main cr3-mediated function...
COST (£) charged to Wellcome (inc VAT when charged)                                            999999.00
Name: 1564, dtype: object

In [245]:
#convert PLoS One costs to numeric & verify
print(df['COST (£) charged to Wellcome (inc VAT when charged)'].dtypes)
plosone = pd.to_numeric(plosone)
print(plosone.dtypes)



object
float64


In [276]:
#remove instances of 999999.00 price
plosone = plosone[plosone!=999999.00]
# use function "summary" on cost series for plosone
plosone.describe()


count       190.000000
mean       1941.483105
std       13909.696072
min         122.310000
25%         850.595000
50%         895.515000
75%        1040.577500
max      192645.000000
Name: COST (£) charged to Wellcome (inc VAT when charged), dtype: float64

In [285]:
summary(plosone)


,Values
Mean,1941.48
Median,895.515
Variance,193479644.81
Std Dev,13909.7


In [288]:
def clean(name):
    

Empty DataFrame
Columns: [PMID/PMCID, Publisher, Journal title, Article title, COST (£) charged to Wellcome (inc VAT when charged)]
Index: []
